In [23]:
import pandas as pd
print (pd.__version__)
import numpy as np
import matplotlib as plt
import os

2.2.3


In [ ]:
stock_names = ['aal', 'aapl', 'adbe', 't', 'tmus']
folder = 'stock_market_dataset'

def create_multi_indexed_df(stock_names, folder):       # self-explanatory function.
    all_stocks = pd.DataFrame()         # create empty df to store values.
    
    # go over all stocks
    for stock in stock_names:
        
        file_path = os.path.join(folder, (stock + '.us.csv'))
        df = pd.read_csv(file_path)
        df['Ticker'] = stock        # add this column to create multi-index later.
        
        all_stocks = pd.concat(     # add stock data to df
            [all_stocks, df], 
            ignore_index = True
        )

    all_stocks['Date'] = pd.to_datetime(all_stocks['Date'])     # convert dates to datetime
    all_stocks = all_stocks.set_index(['Ticker', 'Date'])       # create multi-index
    
    return all_stocks

df = create_multi_indexed_df(stock_names, folder)
print(df)

                     Open    High     Low   Close    Volume  OpenInt
Ticker Date                                                         
aal    2013-12-10  23.698  24.345  23.610  24.064  18043837        0
       2013-12-11  24.607  26.288  24.539  25.139  43224764        0
       2013-12-12  25.342  25.832  24.616  24.616  20749989        0
       2013-12-13  24.771  25.437  24.684  25.369  12588700        0
       2013-12-16  25.749  25.888  25.485  25.739  17136729        0
...                   ...     ...     ...     ...       ...      ...
tmus   2017-11-06  56.200  56.500  54.930  55.540  16981806        0
       2017-11-07  55.780  56.030  54.600  55.350   8785391        0
       2017-11-08  55.360  56.415  55.240  56.220   5977832        0
       2017-11-09  56.500  56.569  55.630  56.000   5204316        0
       2017-11-10  55.690  57.040  55.540  56.800   5758882        0

[28293 rows x 6 columns]


In [25]:
print(df.index.get_level_values('Date'))
print(df.loc['aal'])

DatetimeIndex(['2013-12-10', '2013-12-11', '2013-12-12', '2013-12-13',
               '2013-12-16', '2013-12-17', '2013-12-18', '2013-12-19',
               '2013-12-20', '2013-12-23',
               ...
               '2017-10-30', '2017-10-31', '2017-11-01', '2017-11-02',
               '2017-11-03', '2017-11-06', '2017-11-07', '2017-11-08',
               '2017-11-09', '2017-11-10'],
              dtype='datetime64[ns]', name='Date', length=28293, freq=None)
              Open    High     Low   Close    Volume  OpenInt
Date                                                         
2013-12-10  23.698  24.345  23.610  24.064  18043837        0
2013-12-11  24.607  26.288  24.539  25.139  43224764        0
2013-12-12  25.342  25.832  24.616  24.616  20749989        0
2013-12-13  24.771  25.437  24.684  25.369  12588700        0
2013-12-16  25.749  25.888  25.485  25.739  17136729        0
...            ...     ...     ...     ...       ...      ...
2017-11-06  47.356  47.995  47.177  47

In [26]:
def fill_missing_dates(df):
    stocks = df.index.get_level_values('Ticker').unique()
    res = pd.DataFrame()
    for stock in stocks:
        df1 = df.loc[stock]
        # print(stock, df1)
        all_dates = pd.date_range(
            df1.index.get_level_values('Date').min(),
            df1.index.get_level_values('Date').max()
        )
        df1 = df1.reindex(all_dates)
        # print(df1)
        df1['Ticker'] = stock
        df1 = df1.reset_index(names = ['Date'])
        # print(df1)
        res = pd.concat([res, df1], ignore_index=True)
    
    res = res.set_index(['Ticker', 'Date'])
    res = res.interpolate()

    return res
        
dff = fill_missing_dates(df)
print(dff)
for i in stock_names:
    print(i, dff.loc[i])
# print(df)


                     Open       High     Low      Close        Volume  OpenInt
Ticker Date                                                                   
aal    2013-12-10  23.698  24.345000  23.610  24.064000  1.804384e+07      0.0
       2013-12-11  24.607  26.288000  24.539  25.139000  4.322476e+07      0.0
       2013-12-12  25.342  25.832000  24.616  24.616000  2.074999e+07      0.0
       2013-12-13  24.771  25.437000  24.684  25.369000  1.258870e+07      0.0
       2013-12-14  25.097  25.587333  24.951  25.492333  1.410471e+07      0.0
...                   ...        ...     ...        ...           ...      ...
tmus   2017-11-06  56.200  56.500000  54.930  55.540000  1.698181e+07      0.0
       2017-11-07  55.780  56.030000  54.600  55.350000  8.785391e+06      0.0
       2017-11-08  55.360  56.415000  55.240  56.220000  5.977832e+06      0.0
       2017-11-09  56.500  56.569000  55.630  56.000000  5.204316e+06      0.0
       2017-11-10  55.690  57.040000  55.540  56.800

In [27]:
def drop_old_values(df, yrs):
    cutoff = pd.Timestamp.today() - pd.DateOffset(years = yrs)
    res = df[df.index.get_level_values('Date') >= cutoff]

    return res

dff_recent = drop_old_values(dff, 10)
print(dff_recent)

                     Open       High        Low      Close      Volume  \
Ticker Date                                                              
aal    2015-06-02  42.264  43.046000  42.012000  42.733000  15193600.0   
       2015-06-03  42.314  42.860000  41.982000  42.100000  13597513.0   
       2015-06-04  42.022  42.930000  40.879000  41.161000  14473964.0   
       2015-06-05  41.123  41.563000  40.713000  40.723000  10194780.0   
       2015-06-06  40.778  41.126333  39.919333  40.117667  15592669.0   
...                   ...        ...        ...        ...         ...   
tmus   2017-11-06  56.200  56.500000  54.930000  55.540000  16981806.0   
       2017-11-07  55.780  56.030000  54.600000  55.350000   8785391.0   
       2017-11-08  55.360  56.415000  55.240000  56.220000   5977832.0   
       2017-11-09  56.500  56.569000  55.630000  56.000000   5204316.0   
       2017-11-10  55.690  57.040000  55.540000  56.800000   5758882.0   

                   OpenInt  
Ticker D

In [28]:
def daily_returns(df, column):
    for stock in stock_names:
        returns_col = [0]
        req_col = df.loc[stock][column].values
        for date in range(len(req_col)-1):
            returns_col.append(req_col[date+1]/req_col[date] - 1)
        
        df.loc[(stock, slice(None)), f'{column}_daily_returns'] = returns_col
    return df

print(daily_returns(dff_recent, 'Close'))


                     Open       High        Low      Close      Volume  \
Ticker Date                                                              
aal    2015-06-02  42.264  43.046000  42.012000  42.733000  15193600.0   
       2015-06-03  42.314  42.860000  41.982000  42.100000  13597513.0   
       2015-06-04  42.022  42.930000  40.879000  41.161000  14473964.0   
       2015-06-05  41.123  41.563000  40.713000  40.723000  10194780.0   
       2015-06-06  40.778  41.126333  39.919333  40.117667  15592669.0   
...                   ...        ...        ...        ...         ...   
tmus   2017-11-06  56.200  56.500000  54.930000  55.540000  16981806.0   
       2017-11-07  55.780  56.030000  54.600000  55.350000   8785391.0   
       2017-11-08  55.360  56.415000  55.240000  56.220000   5977832.0   
       2017-11-09  56.500  56.569000  55.630000  56.000000   5204316.0   
       2017-11-10  55.690  57.040000  55.540000  56.800000   5758882.0   

                   OpenInt  Close_dai

C:\Users\Admin\AppData\Local\Temp\ipykernel_22172\1133870374.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[(stock, slice(None)), f'{column}_daily_returns'] = returns_col


In [29]:
def moving_avg(df, days, column):
    for stock in stock_names:
        avg_col = []
        req_col = df.loc[stock][column].values
        for date in range(len(req_col)):
            window = req_col[max(0, date - days + 1) : date + 1]
            moving_avg = np.mean(window)    # use np.nanmean if df isn't clean.
            avg_col.append(moving_avg)
        
        df.loc[(stock, slice(None)), f'{column}_{days}_day_avg'] = avg_col
    return df

print(moving_avg(dff_recent, 7, 'Close'))   
print(moving_avg(dff_recent, 30, 'Close'))

                     Open       High        Low      Close      Volume  \
Ticker Date                                                              
aal    2015-06-02  42.264  43.046000  42.012000  42.733000  15193600.0   
       2015-06-03  42.314  42.860000  41.982000  42.100000  13597513.0   
       2015-06-04  42.022  42.930000  40.879000  41.161000  14473964.0   
       2015-06-05  41.123  41.563000  40.713000  40.723000  10194780.0   
       2015-06-06  40.778  41.126333  39.919333  40.117667  15592669.0   
...                   ...        ...        ...        ...         ...   
tmus   2017-11-06  56.200  56.500000  54.930000  55.540000  16981806.0   
       2017-11-07  55.780  56.030000  54.600000  55.350000   8785391.0   
       2017-11-08  55.360  56.415000  55.240000  56.220000   5977832.0   
       2017-11-09  56.500  56.569000  55.630000  56.000000   5204316.0   
       2017-11-10  55.690  57.040000  55.540000  56.800000   5758882.0   

                   OpenInt  Close_dai

C:\Users\Admin\AppData\Local\Temp\ipykernel_22172\1324256139.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[(stock, slice(None)), f'{column}_{days}_day_avg'] = avg_col
C:\Users\Admin\AppData\Local\Temp\ipykernel_22172\1324256139.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[(stock, slice(None)), f'{column}_{days}_day_avg'] = avg_col


In [30]:
def rolling_volatility(df, days, column):
    for stock in stock_names:
        std_dev_col = []
        req_col = df.loc[stock][column].values
        for date in range(len(req_col)):
            window = req_col[max(0, date - days + 1) : date + 1]
            std_dev = np.std(window)
            std_dev_col.append(std_dev)

        df.loc[(stock, slice(None)), f'{column}_{days}_day_volatility'] = std_dev_col

    return df

print(rolling_volatility(dff_recent, 30, 'Close'))

                     Open       High        Low      Close      Volume  \
Ticker Date                                                              
aal    2015-06-02  42.264  43.046000  42.012000  42.733000  15193600.0   
       2015-06-03  42.314  42.860000  41.982000  42.100000  13597513.0   
       2015-06-04  42.022  42.930000  40.879000  41.161000  14473964.0   
       2015-06-05  41.123  41.563000  40.713000  40.723000  10194780.0   
       2015-06-06  40.778  41.126333  39.919333  40.117667  15592669.0   
...                   ...        ...        ...        ...         ...   
tmus   2017-11-06  56.200  56.500000  54.930000  55.540000  16981806.0   
       2017-11-07  55.780  56.030000  54.600000  55.350000   8785391.0   
       2017-11-08  55.360  56.415000  55.240000  56.220000   5977832.0   
       2017-11-09  56.500  56.569000  55.630000  56.000000   5204316.0   
       2017-11-10  55.690  57.040000  55.540000  56.800000   5758882.0   

                   OpenInt  Close_dai

C:\Users\Admin\AppData\Local\Temp\ipykernel_22172\2005244452.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[(stock, slice(None)), f'{column}_{days}_day_volatility'] = std_dev_col
